## 문자 문제

유니코드 표준은 문자의 단위 원소와 특정 바이트 표현을 명확히 구분한다.

문자를 표현하는 실제 바이트는 사용하는 인코딩에 따라 달라진다.
인코딩은 코드 포인트를 바이트 시퀀스로 변환하는 알고리즘이다.

코드 포인트를 바이트로 변환하는 것을 **인코딩**, 바이트를 코드 포인트로 변환하는 것을 **디코딩**이라고 한다.

In [ ]:
s = 'café'
len(s)

b = s.encode('utf8')
print(b)
len(b)
b.decode('utf8')

bytes 시퀀스는 알아보기 어려운 기계 메모리 덤프로, 유니코드 str은 '사람'이 읽을 수 있는 텍스트로 생각하면 된다. 그러면 bytes 시퀀스를 사람이 읽을 수 있는
텍스트로 **디코딩(해독)** 하고, str을 저장하거나 전송하기 위해 bytes로 **인코딩(암호화)** 한다는 말이 이해될 것이다.

## 바이트에 대한 기본 지식
bytes 형은 파이썬 3에서 소개된 불변형이고, bytearray는 파이썬 2.6에 추가된 가변형이다
bytes와 bytearray에 들어 있는 각 항목은 0에서 255 사이의 정수로, 파이썬 2의 str에 들어있는 문자 하나로 구성된 문자열과는 다르다.